#  **Colab File for CNN training- *PyTorch***
Before you start please prepare the following:


1.   Uplaod the dataset as Zip file to GDrive
2.   Upload code files to GDrive



**Please MAKE sure that you are using GPU . Go to Runtime>> Change Runtime type  and select GPU**

#  **Connect to Google Drive**
 Allow Google Colab to connect to your GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import clear_output

2- Change directory to where you want to keep your dataset

3 - Download Dataset

In [ ]:
# Download Dataset
from IPython.display import clear_output
# !pip install --upgrade --no-cache-dir gdown
#!gdown 1oKH1urVUELrrnQfDTPDt4Aj5H3NwNzn6 #0.4451, 0.4431, 0.4012 - 0.2162, 0.2093, 0.2245
!gdown 1swRiQZRCU3tjYotRIfdWSqSVVGN9RCoa

!jar xvf 'Data.zip'
clear_output()

In [ ]:
!rm -r 'Data.zip'

4 - Download Codes

In [ ]:
%cd /content/

In [ ]:
#!gdown --id 1GJc1zBMaY5tirewUOrKQcGv43SoaIVWS #1.6
!gdown 1rE78950bEEQd_1WZ80TUXPwcFj-zSyhV #2.0
!jar xvf '2D_Segmentation.zip'
clear_output()

In [ ]:
!rm -r '2D_Segmentation.zip'

# Install and Import Required Libraries (Conditionally Optional)
Use 'pip' to install libraries. Put '!' i.e. Exclamation Mark before each command, exclusive to Jupyter Notebooks. Some are already installed by Google Colab and others are required. If the IMPORT statements are present in the python code being run, no need to follow this step.

In [ ]:
!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116

#!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
clear_output() #comment out to debug

# CudaVersion = !nvcc --version
# CudaVersion = CudaVersion[3].partition('V')[2][0:4]
# if CudaVersion == '11.0':
#   # CUDA 11.0
#   !pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
# elif CudaVersion == '10.2':
#   # CUDA 10.2
#   !pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2
# elif CudaVersion == '10.1':
#   # CUDA 10.1
#   !pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
# else:
#   # CUDA 9.2
#   !pip install torch==1.7.1+cu92 torchvision==0.8.2+cu92 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# !pip install torch
# !pip install torchvision
# !pip install torchsummary
# !pip install torch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0
!pip install numpy
!pip install pandas
!pip install -U scikit-learn
!pip install scikit-image
!pip install ipython

!pip install Pillow
!pip install matplotlib
!pip install seaborn
!pip install gdown

!pip install git+https://github.com/qubvel/segmentation_models.pytorch
!pip install segmentation-models-pytorch
clear_output() #comment out to debug

In [ ]:
# Printing out all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# PyTorch
import torch
from torchvision import transforms, models
from torch import optim, cuda, tensor
from torch.utils.data import DataLoader
import torch.nn as nn
# warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
# Data science tools
import numpy as np
import os
from os import path
from importlib import import_module
# Visualizations
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14
# customized functions

# GPU Checking (Optional)
Check the available GPU from the Google Server, GPU model and other related information.

In [ ]:
import torch
print("Is CUDA enabled GPU Available?", torch.cuda.is_available())
print("Is GPU Initialized yet?", torch.cuda.is_initialized())
print("GPU Number:", torch.cuda.device_count())
print("Current GPU Index:", torch.cuda.current_device())
print("GPU Type:", torch.cuda.get_device_name(device=None))
print("GPU Capability:", torch.cuda.get_device_capability(device=None))

# Available Models

**SMP Loss List:**

https://smp.readthedocs.io/en/latest/losses.html

**SMP Encoder List:**

https://smp.readthedocs.io/en/latest/encoders.html

https://smp.readthedocs.io/en/latest/encoders_timm.html

**SMP Decoder List:**

https://smp.readthedocs.io/en/latest/models.html

**Additional Decoders**

* 'Unet3Plus'

**ONN-Based Decoders**

* 'SelfONN_Unet'

* 'SelfONN_UnetPlusPlus'

* 'SelfONN_ResUnet'

* 'SelfONN_FPN'

**Custom Models**

* 'UNet'

* 'M_UNet'

* 'UNet_2Plus'

* 'UNet_3Plus'

* 'MultiResUNet'

* 'CSNet'

* 'Self_UNet'

* 'SelfUNet_compact'

* 'SelfUNet_super_compact'

* 'Super_FPN'

* 'Attention_Super_FPN'

* 'SA_UNet'

* 'ACC_UNet'

# **Model Training**



## Multi-class Augmentation

In [ ]:
from PIL import Image
import cv2
import os
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

#palette = [155, 38, 182, 14, 135, 204, 124, 252, 0, 255, 20, 147, 169, 169, 169, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15]
palette = [0,0,0,71,39,119,62,73,137,48,103,141,37,130,142,30,157,136,53,183,120,109,206,88,181,221,43,253,231,36]
images_location = r'/content/Data/Train'

max_train = 2000

transformations = A.Compose([
    #A.Rotate(limit = [30,30],p=1,border_mode=cv2.BORDER_CONSTANT),
    #A.VerticalFlip(p = 0.5),
    #A.Resize(height, width, interpolation=cv2.INTER_AREA, always_apply=False, p=1),
    A.HorizontalFlip(p = 0.5),
    A.Affine(translate_percent = {'y' : (-0.3, -0.1)}, p = 1,mode=cv2.BORDER_CONSTANT),
    #A.Affine(translate_percent = {'y' : (-0.05, 0.05)}, p = 1,mode=cv2.BORDER_CONSTANT),
    A.Affine(translate_percent = {'x' : (-0.05, 0.05)}, p = 0.5,mode=cv2.BORDER_CONSTANT),
    A.Affine(shear = [-2,2], p = 0.5, mode=cv2.BORDER_CONSTANT),
    A.Rotate(limit = [-20,20],p=1, border_mode=cv2.BORDER_CONSTANT),
    A.Affine(scale = (0.95, 1.05), p = 0.5, keep_ratio = True, mode=cv2.BORDER_CONSTANT),

    #A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.05, rotate_limit=10, border_mode=cv2.BORDER_CONSTANT,always_apply=False,p=1),
    #A.Affine(shear = [30,30], p = 0.5, mode=cv2.BORDER_REFLECT_101),
    #A.RandomGridShuffle (grid=(2, 2), always_apply=False, p = 0.5),
    #A.RandomScale(scale_limit=0.1, interpolation=cv2.INTER_CUBIC, always_apply=False, p=1)
])

for folds in os.listdir(images_location):

  fold_path = os.path.join(images_location, folds)
  image_list  = os.listdir(os.path.join(fold_path,"images"))
  image_list = np.random.choice(image_list, int(len(image_list)), replace = False)
  remain_image = max_train - int(len(image_list))

  for i in tqdm(range(remain_image)):

    image_name = image_list[i % len(image_list)]
    image_path = os.path.join(fold_path,"images",image_name)

    image = cv2.imread(image_path)
    mask_name = image_name
    mask_path = os.path.join(fold_path,"masks",mask_name)
    mask = Image.open(mask_path)
    mask = np.array(mask)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    transformed = transformations(image = image, mask = mask)
    processed_image = transformed['image']
    processed_mask = transformed['mask']

    file_name = os.path.join(fold_path,"images",image_name[:-4]+f'_aug{i}.png')

    cv2.imwrite(file_name, processed_image)


    file_name = os.path.join(fold_path,"masks",image_name[:-4]+f'_aug{i}.png')


    processed_mask = Image.fromarray(processed_mask)

    processed_mask = processed_mask.convert('P')

    processed_mask.putpalette(palette)

    processed_mask.save(file_name)


## Training Setup

In [ ]:
# Dependencies directory
%cd /content/2D_Segmentation

In [ ]:
from image_mean_std import cal_dir_stat
in_channel_num = 3
image_dir = '/content/Data/Train/fold_1/images'

mean, std = cal_dir_stat(image_dir, in_channel_num)
print(f'mean: {mean}\n', f'std: {std}')

In [ ]:
%%writefile config.py
# CNN configuration file

##### DO NOT EDIT THESE LINES #####
config = {}
###################################


#### START EDITING FROM HERE ######
config['parentdir'] = '/content/'                                                        # main directory
config['ONN'] = False                                                                     # set to 'True' if you are using ONN
config['batch_size'] = 16                                                                # batch size, Change to fit hardware
config['in_channels'] = 1                                                                # 1 for gray scale x-rays, and 3 for RGB (3channel) x-rays
config['out_channels'] = 5                                                               # number of classes (Use 1 for binary, Use total number of classes with background for multiclass)
config['palette'] = [0,0,0,71,39,119,62,73,137,48,103,141,37,130,142,30,157,136,53,183,120,109,206,88,181,221,43,253,231,36]
config['disregard_background'] = True                                                    # For Multi-Class Only : Whether to Disregard Background (0 Class) in metric calculation
config['input_mean'] = [0.1455]                                             # provide 3 numbers for RGB images or 1 number for gray scale images in list format
config['input_std'] = [0.1797]                                             # provide 3 numbers for RGB images or 1 number for gray scale images in list format
config['optim_fc'] = 'Adam'                                                              # 'Adam' or 'SGD'
config['lr'] = 1e-3                                                                      # learning rate
config['class_weights'] = None                                                           # class weights for multi class masks, default: none
config['lossType'] = 'SMP_DiceLoss'                                                          # loss function: 'CrossEntropy' for multi-class. 'BCELoss' or 'DiceLoss' for binary class
config['n_epochs']  = 100                                                               # number of training epochs
config['epochs_patience'] = 5                                                            # if val loss did not decrease for a number of epochs then decrease learning rate by a factor of lr_factor
config['lr_factor'] = 0.2
config['max_epochs_stop'] = 20                                                            # maximum number of epochs with no improvement in validation loss for early stopping
config['num_folds']  = 1                                                                 # number of cross validation folds
config['Resize_h'] = 256                                                                 # network input size
config['Resize_w'] = config['Resize_h']
# config['load_model'] = config['parentdir'] + 'load_model/Densenet121_FPN_lung_seg_fold_1.pt'    # specify path of pretrained model wieghts or set to False to train from scratch
config['load_model'] = False                                                             # specify path of pretrained model wieghts or set to False to train from scratch
config['Test_Mask'] = True                                                               # set to true if you have the test masks, to compute Evaluation metricies over test set
config['model_type'] = 'Custom'                                                             # SMP libary models : SMP | ONN-Based Decoders : ONN_dec | Custom models : Custom
config['model_to_load'] = 'UNet'                                     # enter model name like this: <encoder_name>*<decoder_name> (names should match the documentation exactly)
config['model_name'] = 'MulticlassTest_Promit_UNet'                              # choose a unique name for result folder
config['decoder_attention'] = None                                                  # Turn on Attention Layer in Unet/Unet++ (None / 'scse') (only works with SMP and ONN_dec)
config['encoder_depth']  = 5                                                            # number of encoder layers (For pretrained weights default: 5) (not usable with PAN decoder)
config['encoder_weights'] = 'imagenet'                                                  # pretrained weights: 'imagenet' | Train from scratch: None (some encoders have multiple weights check documentation)
config['activation'] = None                                                             # last layer activation function (default: None | Available functions: provided in the list above)
config['q_order'] = 3                                                                    # ONN q-order
config['max_shift'] = 0                                                                  # ONN max_shift
config['seg_threshold'] = 0.5                                                           # Segmentation Threshold (Default 0.5)
config['U_init_features'] = 32                                                           # Only for model_type : Custom | number of kernals in the first UNet conv layer ('32' & '64' are common values)
config['fold_to_run'] = [1,1] # define as [] to loop through all folds, or specify start and end folds i.e. [3, 5] or [5, 5]
config['Results_path'] = "/content/drive/MyDrive/Rusab/Multiclass_test/" + "Results"   # main results file
config['save_path'] = config['Results_path'] +'/'+ config['model_name']                  # save path
config['generated_masks']  =  '/content/drive/MyDrive/Rusab/Multiclass_test/' + 'Generated_mask'         # path to save generated_masks for test set
##################

2- Start training

In [ ]:
#!rm -r /content/Data/Val/fold_1/.ipynb_checkpoints

In [ ]:
!python TrainUnet.py

3-Custom evaluation

In [ ]:
import metric_evaluation_opt
gnd = '/content/Data/Test'
predict = '/content/drive/MyDrive/Rusab/Multiclass_test/Generated_mask/BinaryclassTest_Carotid_resnet18*SelfONN_Unet'
class_map = []
metric_evaluation_opt.evaluation(gnd, predict, save = True, reduction = 'micro', classes = 1, class_map = class_map, disregard_background = True)

#**Run your model on an exteranl dataset or re-generate the test results**


In [ ]:
%cd /content/2D_Segmentation

In [ ]:
#/content/drive/MyDrive/Rusab/Coronary_segmentation/
%%writefile config_test.py
# CNN test configuration file

##### DO NOT EDIT THESE LINES #####
config = {}
###################################

#### START EDITING FROM HERE ######
config['parentdir'] = '/content/'                                                        # main directory
config['ONN'] = True                                                                     # set to 'True' if you are using ONN
config['batch_size'] = 16                                                                # batch size, Change to fit hardware
config['in_channels'] = 3                                                                # 1 for gray scale x-rays, and 3 for RGB (3channel) x-rays
config['out_channels'] = 5                                                               # number of classes (Use 1 for binary, Use total number of classes with background for multiclass)
config['palette'] = [155, 38, 182, 14, 135, 204, 124, 252, 0, 255, 20, 147, 169, 169, 169, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15]
config['disregard_background'] = False                                                    # For Multi-Class Only : Whether to Disregard Background (0 Class) in metric calculation
config['input_mean'] = [0.4451, 0.4431, 0.4012]                                             # provide 3 numbers for RGB images or 1 number for gray scale images in list format
config['input_std'] = [0.2162, 0.2093, 0.2245]                                             # provide 3 numbers for RGB images or 1 number for gray scale images in list format
config['num_folds'] = 1                                                                  # number of folds
config['class_weights'] = None                                                           # class weights for multi class masks, default: none
config['lossType'] = 'SMP_DiceLoss'
config['Resize_h'] = 256                                                                 # network input size
config['Resize_w'] = config['Resize_h']
# config['load_model'] = '/content/drive/Shareddrives/Rusab/Coronary_Angiogram/Results/MixTrain_densenet121*SelfONN_ResUnet/MixTrain_densenet121*SelfONN_ResUnet_fold_3.pt'        # specify full path of pretrained model pt file
config['load_model'] = False                                                             # specify path of pretrained model wieghts or set to False to train from scratch
config['Test_Mask'] = True                                                               # set to true if you have the test masks, to compute Evaluation metricies over test set
config['model_name'] = 'MulticlassTest_UNet_sanity'            # choose a unique name for result folder
config['decoder_attention'] = 'scse'                                                   #decoder attention type
config['seg_threshold'] = 0.5                                                           # Segmentation Threshold (Default 0.5)
config['new_name'] = 'MulticlassTest_UNet_sanity'                                                                 #specify a new folder name to save test results,
config['fold_to_run'] = [1,1] # define as [] to loop through all folds, or specify start and end folds i.e. [3, 5] or [5, 5]     # else set to False to overwrite test results genertaed by train code
##################

##################
config['generated_masks']  = "/content/drive/MyDrive/Rusab/Multiclass_test/" + 'Generated_mask'        # path to save generated_masks for test set
config['Results_path'] = "/content/drive/MyDrive/Rusab/Multiclass_test/" + "Results"                      # main results file
if config['new_name']:
    config['save_path'] = config['Results_path'] +'/'+ config['new_name']                                      # new save path
else:
    config['save_path'] = config['Results_path'] +'/'+ config['model_name']                                    # same save path used for training
##################


Test on Dataset

In [ ]:
!python TestUnet.py

In [ ]:
import metric_evaluation_opt
gnd = '/content/Data/Test'
predict = '/content/drive/MyDrive/Rusab/Multiclass_test/Generated_mask/BinaryclassTest_Carotid_resnet18*SelfONN_Unet'
class_map = []
metric_evaluation_opt.evaluation(gnd, predict, save = True, reduction = 'micro', classes = 1, class_map = class_map, disregard_background = True)

Colorize Predictions

In [2]:
seg_type = 'binary' #'multiclass' for multiclass
predict_dir = '/content/drive/MyDrive/Coronary_Angiogram/colabpro3/Generated_mask/FinalOldMask_mit_b1*FPN'
color_output_dir = predict_dir + '/compare'

for i in range(1,6):
  gnd = '/content/Data/Test/fold_'+ str(i) +'/masks'
  predict = predict_dir + '/fold_' + str(i)
  des = color_output_dir  + '/fold_'+ str(i)
  

  !python colorize_evaluation.py --gt $gnd --pred $predict --des $des --type $seg_type

usage: colorize_evaluation.py [-h] --gt GT --pred PRED --des DES [--type TYPE]
colorize_evaluation.py: error: unrecognized arguments: Works\DroneData_Kaggle\Promit\v1.1.1\Data\Test\fold_1\masks Works\DroneData_Kaggle\Promit\Generated_Masks\MulticlassTest_Promit_densenet121_SelfONN_Unet/fold_1 Works\DroneData_Kaggle\Promit\Generated_Masks\MulticlassTest_Promit_densenet121_SelfONN_Unet/compare/fold_1
